# BETA_random_eqrthquake_locations_w_pyvista_vtk

Testing the random picking of subsurface eqrthquake locations (moment-tensor locations) and 3D plotting of the picks and model 

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 

from sys import argv

from matplotlib.colors import Normalize
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import compress_gm3d_to_file
from pyaspect.model.gm3d_utils import decompress_gm3d_from_file
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import StationHeader
from pyaspect.specfemio.headers import SolutionHeader
from pyaspect.specfemio.headers import CMTSolutionHeader
from pyaspect.specfemio.headers import ForceSolutionHeader
from pyaspect.specfemio.headers import RecordHeader
from pyaspect.specfemio.write import write_cmtsolution
from pyaspect.specfemio.write import write_forcesolution
from pyaspect.specfemio.write import write_grouped_forcesolutions
from pyaspect.specfemio.write import write_stations
from pyaspect.specfemio.write import write_record
from pyaspect.specfemio.write import write_records
from pyaspect.specfemio.read import read_stations
from pyaspect.specfemio.read import read_solution
from pyaspect.specfemio.read import read_cmtsolution
from pyaspect.specfemio.read import read_forcesolution
from pyaspect.specfemio.utils import *

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir} 

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = [0,0,-vslice_gm3d.get_gorigin()[2]]
#nam_origin = list(vslice_gm3d.get_gorigin())
#nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

## create random virtual source (to specfem stations, but using reciprocity -- sources)

In [ ]:
#coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords = vslice_gm3d.getLocalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


n_rand_p = 1000

#stay away from the edges of the model for derivatives 
# and to avoid boundary effects
xy_pad = 1000 

lrx = np.min(xc) + xy_pad
lry = np.min(yc) + xy_pad
lrz = -4300.0

hrx = np.max(xc) - xy_pad
hry = np.max(yc) - xy_pad
hrz = -3000.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

r_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    rz = lrz + srz*np.random.rand()
    r_xyz_list.append([rx,ry,rz])
    
r_xyz = np.array(r_xyz_list)
    

#r_xyz = np.vstack(np.meshgrid(rx,ry,rz)).reshape(3,-1).T
print('r_xyz:\n',r_xyz)


In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

In [ ]:
!pwd

In [ ]:
#!mkdir {data_out_dir}/tmp
!ls {data_out_dir}

## make cross (half or full) group for calculating spacial derivatives

In [ ]:
if 'l_grp_stations' in locals() or 'l_grp_stations' in globals():
    print('deleting')
    del l_grp_stations
    
    
# this is the path to the project dir on the cluster
my_proj_dir = '/scratch/seismology/tcullison/test_mesh/Batch_Src_Test'

    
# make a list of station headers for each random location
l_stations = []
for i in range(len(r_xyz)):
    
    #name = 't' + str(i).zfill(len(str(len(r_xyz))))
    tr_bname = 'tr'
    new_s = StationHeader(name=tr_bname,
                          network='NL',
                          lat_yc=r_xyz[i,1],
                          lon_xc=r_xyz[i,0],
                          elevation=0.0,
                          depth=-r_xyz[i,2],
                          trid=i)
    l_stations.append(new_s)
#print('len(l_stats):',len(l_stations))
                                           
# make the group membors for each station above
# function below returns a list[list[]] like structure
m_delta = 250.0 # distance between cross stations for derivatives
assert m_delta < xy_pad #see cells above this is padding
l_grp_stations = make_grouped_half_cross_station_headers(stations=l_stations,delta=m_delta)
name_list = []
for s in flatten_grouped_headers(l_grp_stations):
    name_list.append(f's{str(s.sid).zfill(2)}g{str(s.gid).zfill(2)}t{str(s.trid).zfill(6)}')
    
print('len(name_list):',len(name_list))
print('len(name_set): ',len(set(name_list)))
    

# this is a list[] structure with unique stations (small chance a group member has same coordinate)
s_grp_stations = sorted(copy.deepcopy(flatten_grouped_headers_unique(l_grp_stations)))

# if lengths are the same then all group members are unique
print(f'len(l_grp): {len(flatten_grouped_headers(l_grp_stations))}')
print(f'len(s_grp): {len(s_grp_stations)}')

#flatten the l_grp_stations (NOT guarantied unique!)
l_grp_stations = sorted(flatten_grouped_headers(l_grp_stations))

In [ ]:
write_stations(data_out_dir + '/tmp',l_grp_stations,auto_name=True,auto_network=True)

# For records only!
#write_stations(data_out_dir + '/tmp',l_grp_stations,write_h=False) 

In [ ]:
!ls -ltrh {data_out_dir + '/tmp'} 

In [ ]:
!head {data_out_dir}/tmp/STATIONS

In [ ]:
!tail {data_out_dir}/tmp/STATIONS

In [ ]:
fqp = data_out_dir + '/tmp'
rw_stations = sorted(read_stations(fqp))

print('stations equal?:', rw_stations == l_grp_stations)

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(rw_stations)
all_g_xyz[:,2] *= -1 #pyview z-up positive and oposize sign of standard geophysics 
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.75)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## make random virtual recievers locations (solutions/sources in specfem)

In [ ]:
#coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords = vslice_gm3d.getLocalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


n_rand_p = 5
rz = -500

lrx = np.min(xc)
lry = np.min(yc)

hrx = np.max(xc)
hry = np.max(yc)

srx = hrx - lrx
sry = hry - lry

s_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    s_xyz_list.append([rx,ry,rz])
    
s_xyz = np.array(s_xyz_list)
    

print('s_xyz:\n',s_xyz)


In [ ]:
pv_spoints = pv.wrap(s_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## make force solution headers

In [ ]:
# make a list of station headers for each random location
l_solutions = []
for i in range(len(s_xyz)):
    
    #NOTE!!!! the depth is set to the NEGATIVE (makes it positive) due to sign convention
    new_s = ForceSolutionHeader(ename=f'Event-{str(i).zfill(4)}',
                                lat_yc=s_xyz[i,1],
                                lon_xc=s_xyz[i,0],
                                depth=-s_xyz[i,2],
                                tshift=0.0,
                                date=datetime.datetime.now(),
                                f0=0.0,
                                factor_fs=1,
                                comp_src_EX=1,
                                comp_src_NY=0,
                                comp_src_Zup=0,
                                proj_id=0,
                                eid=i,
                                sid=0)
    l_solutions.append(new_s)

## make solution group

In [ ]:
l_grp_solutions_h = make_grouped_triplet_force_solution_headers(solutions=l_solutions)
grp_s_xyz = get_xyz_coords_from_solution_list(flatten_grouped_headers(l_grp_solutions_h))

for s in flatten_grouped_headers(l_grp_solutions_h):
    print(f'solution:\n{s}')
    print()
print(f'coords:\n{grp_s_xyz}')
print()

In [ ]:
for grp_s in l_grp_solutions_h:
    p = os.path.join(data_out_dir, 'tmp')
    print(p)
    p = os.path.join(p, f'run{str(grp_s[0].eid+1).zfill(4)}')
    print(p)
    p = os.path.join(p, 'DATA')
    print(p)
    #p = f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}'
    #print(p)
    Path(p).mkdir(parents=True, exist_ok=True)

In [ ]:
!ls {data_out_dir}/tmp
!pwd

In [ ]:
for grp_s in l_grp_solutions_h:
    write_grouped_forcesolutions(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}',grp_s)
    #write_grouped_forcesolutions(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}',grp_s,write_h=False)
    #print(f'{data_out_dir}/tmp/run{str(grp_s[0].eid+1).zfill(4)}')
    #for s in grp_s:
        #print(f's:\n{s}')


In [ ]:
!ls -l {data_out_dir}tmp/run*/DATA

## make record from solutions and stations

In [ ]:
# make stations list per solution_group 
poor_l_records = []

for i in range(len(l_grp_solutions_h)):
    grp_s = l_grp_solutions_h[i]
    l_force_stations = []
    for sol in grp_s:
        l_stat = copy.deepcopy(l_grp_stations)
        for s in l_stat:
            s.eid = sol.eid
            s.sid = sol.sid
        l_force_stations += l_stat

    record = RecordHeader(solutions_h=grp_s,stations_h=l_force_stations,rid=i)
    poor_l_records.append(record)

src_grouped_stations = []
for sgrp in l_grp_solutions_h:
    l_rgrp = []
    for s in sgrp:
        l_rgrp.append(copy.deepcopy(l_grp_stations))
    src_grouped_stations.append(l_rgrp)

print(f'len(l_grp_solutions_h):{len(l_grp_solutions_h)}')
print(f'len(l_grp_solutions_h[0]):{len(l_grp_solutions_h[0])}')
print(f'len(l_grp_solutions_h[0][0]):{len(l_grp_solutions_h[0][0])}')
print(f'len(src_grouped_stations):{len(src_grouped_stations)}')
print(f'len(src_grouped_stations[0]):{len(src_grouped_stations[0])}')
l_records = []
l_records = make_records(l_src=l_grp_solutions_h,l_rec=src_grouped_stations)
    
print(f'len(l_records): {len(l_records)}')
print()

for i in range(len(l_records)):
    print(f'rec[{i}] == prec[{i}]: {l_records[i] == poor_l_records[i]}')

print('Print Out of ALL Records\n')
for i in range(len(l_records)):
    r = l_records[i]
    p = poor_l_records[i]
    print(f'Record-{r.rid}:\n{r}')
    print()
    print('---------------------------------------------------------------------------')
    print()

## pickle record list

In [ ]:
f = open(f'{data_out_dir}/tmp/project_record_list','wb')
pickle.dump(l_records,f)
f.close()

!ls -ltrh {data_out_dir}/tmp/project_record_list

In [ ]:
f = open(f'{data_out_dir}/tmp/project_record_list','rb')
dill_l_records = pickle.load(f)
f.close

print('Check all records:')
print()
for i in range(len(dill_l_records)):
    print(f'Dill_Rec == Orig_Rec?: {dill_l_records == l_records}')

## write records and headers in records:

In [ ]:

proj_fqp = f'{data_out_dir}tmp/' 
#write_record(proj_fqp,dill_l_records[0],fname='test_project.record')
write_records(proj_fqp,dill_l_records,fname='test_project_record',auto_name=True,auto_network=True)

In [ ]:
!ls -ltrh data/output/tmp/
print()
!ls -ltrh {data_out_dir}tmp/run*/Data

In [ ]:
for r in dill_l_records:
    print(f'Record:\n{r}')

In [ ]:
from pyaspect.specfemio.utils import _join_path_fname
test_proj_name = 'FirstTestProject'
test_proj_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'
print(f'test_proj_fqp: {test_proj_fqp}')
!ls {test_proj_fqp}

In [ ]:
from pyaspect.project import make_project
#make_records(l_src=l_grp_solutions_h,l_rec=src_grouped_stations)
make_project(test_proj_name,
             test_proj_fqp,
             test_parfile_fqp,
             test_spec_fqp,
             test_pyutils_fqp,
             test_script_fqp,
             l_grp_solutions_h,
             src_grouped_stations)

In [ ]:
ls -ltrh data/output/tmp/TestProjects/FirstTestProject/

In [ ]:
from pyaspect.parfile import change_parameter
from pyaspect.parfile import change_multiple_parameters

parfile_fqp = os.path.join(data_out_dir,'Par_file')
out_parfile_fqp = os.path.join(data_out_dir,'New_Par_file')

with open(parfile_fqp, 'r') as f:
    lines = f.readlines()

il = 0
for line in lines:
    if il == 313:
        break
    print(line)
    il += 1
'''
''';
    
#change_parameter(parfile_fqp,lines,'SAVE_FORWARD',True,out_parfile_fqp=out_parfile_fqp)
par_keys = ['SIMULATION_TYPE','SAVE_FORWARD','NSTEP','DT','MODEL','MIN_ATTENUATION_PERIOD','LOCAL_PATH','SAVE_MESH_FILES']
par_vals = [4,True,8192,0.002,'default',666666666,'./BOOGER/in_my_nose',True]
s_keys_vals = set(zip(par_keys,par_vals))
print(f's_keys_vals:\n{s_keys_vals}')
print(f'd_keys_vals:\n{[(k,v) for k,v in dict(zip(par_keys,par_vals)).items()]}')

change_multiple_parameters(parfile_fqp,s_keys_vals,out_parfile_fqp=out_parfile_fqp)
    

print('\n\n----------------------- changed -------------------------')

with open(out_parfile_fqp, 'r') as f:
    lines = f.readlines()
    
il = 0
for line in lines:
    if il == 313:
        break
    print(line)
    il += 1
'''
''';


In [ ]:
!diff {parfile_fqp} {out_parfile_fqp}

In [ ]:
import json


out_parfile_json = os.path.join(data_out_dir,'Par_file.json')
with open(out_parfile_json, 'w') as f:
    json.dump(lines,f)

In [ ]:
parfile_fqp = os.path.join(data_out_dir,'Par_file')
with open(parfile_fqp, 'r') as f:
    lines = f.readlines()
    
parfile_pickle = pickle.dumps(lines, 0)
parfile_pickle_str = parfile_pickle.decode('ascii')
#parfile_pickle_str = str(parfile_pickle)

#print(f'Parfile Type:\n{type(parfile_pickle)}')
print(f'Parfile String:\n{type(parfile_pickle_str)}')
print(f'Parfile String:\n{parfile_pickle_str.__repr__()}')

new_par_bytes = parfile_pickle_str.encode('ascii')
new_lines = pickle.loads(new_par_bytes)

'''
print('---------- The Winding Road -----------------')
for line in new_lines:
    print(line)
''';




In [ ]:
test_parfile_str = """(lp0\nV#-----------------------------------------------------------\\u000a\np1\naV#\\u000a\np2\naV# Simulation input parameters\\u000a\np3\naV#\\u000a\np4\naV#-----------------------------------------------------------\\u000a\np5\naV\\u000a\np6\naV# forward or adjoint simulation\\u000a\np7\naV# 1 = forward, 2 = adjoint, 3 = both simultaneously\\u000a\np8\naVSIMULATION_TYPE                 = 1\\u000a\np9\naV# 0 = earthquake simulation,  1/2/3 = three steps in noise simulation\\u000a\np10\naVNOISE_TOMOGRAPHY                = 0\\u000a\np11\naVSAVE_FORWARD                    = .false.\\u000a\np12\nag6\naV# solve a full FWI inverse problem from a single calling program with no I/Os, storing everything in memory,\\u000a\np13\naV# or run a classical forward or adjoint problem only and save the seismograms and/or sensitivity kernels to disk (with costlier I/Os)\\u000a\np14\naVINVERSE_FWI_FULL_PROBLEM        = .false.\\u000a\np15\nag6\naV# UTM projection parameters\\u000a\np16\naV# Use a negative zone number for the Southern hemisphere:\\u000a\np17\naV# The Northern hemisphere corresponds to zones +1 to +60,\\u000a\np18\naV# The Southern hemisphere corresponds to zones -1 to -60.\\u000a\np19\naVUTM_PROJECTION_ZONE             = 11\\u000a\np20\naVSUPPRESS_UTM_PROJECTION         = .true.\\u000a\np21\nag6\naV# number of MPI processors\\u000a\np22\naVNPROC                           = 4 \\u000a\np23\nag6\naV# time step parameters\\u000a\np24\naVNSTEP                           = 4096\\u000a\np25\naVDT                              = 0.001\\u000a\np26\nag6\naV#-----------------------------------------------------------\\u000a\np27\naV#\\u000a\np28\naV# LDDRK time scheme\\u000a\np29\naV#\\u000a\np30\naV#-----------------------------------------------------------\\u000a\np31\naVUSE_LDDRK                       = .false.\\u000a\np32\naVINCREASE_CFL_FOR_LDDRK          = .false.\\u000a\np33\naVRATIO_BY_WHICH_TO_INCREASE_IT   = 1.4\\u000a\np34\nag6\naV#-----------------------------------------------------------\\u000a\np35\naV#\\u000a\np36\naV# Mesh\\u000a\np37\naV#\\u000a\np38\naV#-----------------------------------------------------------\\u000a\np39\nag6\naV# Number of nodes for 2D and 3D shape functions for hexahedra.\\u000a\np40\naV# We use either 8-node mesh elements (bricks) or 27-node elements.\\u000a\np41\naV# If you use our internal mesher, the only option is 8-node bricks (27-node elements are not supported).\\u000a\np42\naVNGNOD                           = 8\\u000a\np43\nag6\naV# models:\\u000a\np44\naV# available options are:\\u000a\np45\naV#   default (model parameters described by mesh properties)\\u000a\np46\naV# 1D models available are:\\u000a\np47\naV#   1d_prem,1d_socal,1d_cascadia\\u000a\np48\naV# 3D models available are:\\u000a\np49\naV#   aniso,external,gll,salton_trough,tomo,SEP,coupled,...\\u000a\np50\naVMODEL                           = gll\\u000a\np51\nag6\naV# path for external tomographic models files\\u000a\np52\naVTOMOGRAPHY_PATH                 = ./DATA/tomo_files/\\u000a\np53\naV# if you are using a SEP model (oil-industry format)\\u000a\np54\naVSEP_MODEL_DIRECTORY             = ./DATA/my_SEP_model/\\u000a\np55\nag6\naV#-----------------------------------------------------------\\u000a\np56\nag6\naV# parameters describing the model\\u000a\np57\naVAPPROXIMATE_OCEAN_LOAD          = .false.\\u000a\np58\naVTOPOGRAPHY                      = .false.\\u000a\np59\naVATTENUATION                     = .false.\\u000a\np60\naVANISOTROPY                      = .false.\\u000a\np61\naVGRAVITY                         = .false.\\u000a\np62\nag6\naV# in case of attenuation, reference frequency in Hz at which the velocity values in the velocity model are given (unused otherwise)\\u000a\np63\naVATTENUATION_f0_REFERENCE        = 18.d0\\u000a\np64\nag6\naV# attenuation period range over which we try to mimic a constant Q factor\\u000a\np65\naVMIN_ATTENUATION_PERIOD          = 999999998.d0\\u000a\np66\naVMAX_ATTENUATION_PERIOD          = 999999999.d0\\u000a\np67\naV# ignore this range and ask the code to compute it automatically instead based on the estimated resolution of the mesh (use this unless you know what you are doing)\\u000a\np68\naVCOMPUTE_FREQ_BAND_AUTOMATIC     = .true.\\u000a\np69\nag6\naV# Olsen\'s constant for Q_mu = constant * V_s attenuation rule\\u000a\np70\naVUSE_OLSEN_ATTENUATION           = .false.\\u000a\np71\naVOLSEN_ATTENUATION_RATIO         = 0.05\\u000a\np72\nag6\naV#-----------------------------------------------------------\\u000a\np73\naV#\\u000a\np74\naV# Absorbing boundary conditions\\u000a\np75\naV#\\u000a\np76\naV#-----------------------------------------------------------\\u000a\np77\nag6\naV# C-PML boundary conditions for a regional simulation\\u000a\np78\naV# (if set to .false., and STACEY_ABSORBING_CONDITIONS is also set to .false., you get a free surface instead\\u000a\np79\naV# in the case of elastic or viscoelastic mesh elements, and a rigid surface in the case of acoustic (fluid) elements\\u000a\np80\naVPML_CONDITIONS                  = .false.\\u000a\np81\nag6\naV# C-PML top surface\\u000a\np82\naVPML_INSTEAD_OF_FREE_SURFACE     = .false.\\u000a\np83\nag6\naV# C-PML dominant frequency\\u000a\np84\naVf0_FOR_PML                      = 0.05555\\u000a\np85\nag6\naV# parameters used to rotate C-PML boundary conditions by a given angle (not completed yet)\\u000a\np86\naV# ROTATE_PML_ACTIVATE           = .false.\\u000a\np87\naV# ROTATE_PML_ANGLE              = 0.\\u000a\np88\nag6\naV# absorbing boundary conditions for a regional simulation\\u000a\np89\naV# (if set to .false., and PML_CONDITIONS is also set to .false., you get a free surface instead\\u000a\np90\naV# in the case of elastic or viscoelastic mesh elements, and a rigid surface in the case of acoustic (fluid) elements\\u000a\np91\naVSTACEY_ABSORBING_CONDITIONS     = .true.\\u000a\np92\nag6\naV# absorbing top surface (defined in mesh as \'free_surface_file\')\\u000a\np93\naVSTACEY_INSTEAD_OF_FREE_SURFACE  = .false.\\u000a\np94\nag6\naV# When STACEY_ABSORBING_CONDITIONS is set to .true. :\\u000a\np95\naV# absorbing conditions are defined in xmin, xmax, ymin, ymax and zmin\\u000a\np96\naV# this option BOTTOM_FREE_SURFACE can be set to .true. to\\u000a\np97\naV# make zmin free surface instead of absorbing condition\\u000a\np98\naVBOTTOM_FREE_SURFACE             = .false.\\u000a\np99\nag6\naV#-----------------------------------------------------------\\u000a\np100\naV#\\u000a\np101\naV# undoing attenuation and/or PMLs for sensitivity kernel calculations\\u000a\np102\naV#\\u000a\np103\naV#-----------------------------------------------------------\\u000a\np104\nag6\naV# to undo attenuation and/or PMLs for sensitivity kernel calculations or forward runs with SAVE_FORWARD\\u000a\np105\naV# use the flag below. It performs undoing of attenuation and/or of PMLs in an exact way for sensitivity kernel calculations\\u000a\np106\naV# but requires disk space for temporary storage, and uses a significant amount of memory used as buffers for temporary storage.\\u000a\np107\naV# When that option is on the second parameter indicates how often the code dumps restart files to disk (if in doubt, use something between 100 and 1000).\\u000a\np108\naVUNDO_ATTENUATION_AND_OR_PML     = .false.\\u000a\np109\naVNT_DUMP_ATTENUATION             = 500\\u000a\np110\nag6\naV#-----------------------------------------------------------\\u000a\np111\naV#\\u000a\np112\naV# Visualization\\u000a\np113\naV#\\u000a\np114\naV#-----------------------------------------------------------\\u000a\np115\nag6\naV# save AVS or OpenDX movies\\u000a\np116\naV# MOVIE_TYPE = 1 to show the top surface\\u000a\np117\naV# MOVIE_TYPE = 2 to show all the external faces of the mesh\\u000a\np118\naVCREATE_SHAKEMAP                 = .false.\\u000a\np119\naVMOVIE_SURFACE                   = .false.\\u000a\np120\naVMOVIE_TYPE                      = 2\\u000a\np121\naVMOVIE_VOLUME                    = .false.\\u000a\np122\naVSAVE_DISPLACEMENT               = .true.\\u000a\np123\naVUSE_HIGHRES_FOR_MOVIES          = .false.\\u000a\np124\naVNTSTEP_BETWEEN_FRAMES           = 500\\u000a\np125\naVHDUR_MOVIE                      = 0.0\\u000a\np126\nag6\naV# save AVS or OpenDX mesh files to check the mesh\\u000a\np127\naVSAVE_MESH_FILES                 = .false.\\u000a\np128\nag6\naV# path to store the local database file on each node\\u000a\np129\naVLOCAL_PATH                      = ./OUTPUT_FILES/DATABASES_MPI\\u000a\np130\naV#LOCAL_PATH                      = ./COMMON/DATABASES_MPI\\u000a\np131\nag6\naV# interval at which we output time step info and max of norm of displacement\\u000a\np132\naVNTSTEP_BETWEEN_OUTPUT_INFO      = 1000\\u000a\np133\nag6\naV#-----------------------------------------------------------\\u000a\np134\naV#\\u000a\np135\naV# Sources\\u000a\np136\naV#\\u000a\np137\naV#-----------------------------------------------------------\\u000a\np138\nag6\naV# sources and receivers Z coordinates given directly (i.e. as their true position) instead of as their depth\\u000a\np139\naVUSE_SOURCES_RECEIVERS_Z         = .false.\\u000a\np140\nag6\naV# use a (tilted) FORCESOLUTION force point source (or several) instead of a CMTSOLUTION moment-tensor source.\\u000a\np141\naV# This can be useful e.g. for oil industry foothills simulations or asteroid simulations\\u000a\np142\naV# in which the source is a vertical force, normal force, tilted force, impact etc.\\u000a\np143\naV# If this flag is turned on, the FORCESOLUTION file must be edited by giving:\\u000a\np144\naV# - the corresponding time-shift parameter,\\u000a\np145\naV# - the half duration parameter of the source,\\u000a\np146\naV# - the coordinates of the source,\\u000a\np147\naV# - the magnitude of the force source,\\u000a\np148\naV# - the components of a (non necessarily unitary) direction vector for the force source in the E/N/Z_UP basis.\\u000a\np149\naV# The direction vector is made unitary internally in the code and thus only its direction matters here;\\u000a\np150\naV# its norm is ignored and the norm of the force used is the factor force source times the source time function.\\u000a\np151\naVUSE_FORCE_POINT_SOURCE          = .true.\\u000a\np152\nag6\naV# set to true to use a Ricker source time function instead of the source time functions set by default\\u000a\np153\naV# to represent a (tilted) FORCESOLUTION force point source or a CMTSOLUTION moment-tensor source.\\u000a\np154\naVUSE_RICKER_TIME_FUNCTION        = .false.\\u000a\np155\nag6\naV# Use an external source time function.\\u000a\np156\naV# if .true. you must add a file with your source time function and the file name\\u000a\np157\naV# path relative to lauching directory at the end of CMTSOLUTION or FORCESOURCE file\\u000a\np158\naV# (with multiple sources, one file per source is required).\\u000a\np159\naV# This file must have a single column containing the amplitude of the source at that time step,\\u000a\np160\naV# and on its first line it must contain the time step used, which must be equal to DT as defined at the beginning of this Par_file (the code will check that).\\u000a\np161\naV# Be sure when this option is .false. to remove the name of stf file in CMTSOLUTION or FORCESOURCE\\u000a\np162\naVUSE_EXTERNAL_SOURCE_FILE        = .false.\\u000a\np163\nag6\naV# print source time function\\u000a\np164\naVPRINT_SOURCE_TIME_FUNCTION      = .true.\\u000a\np165\nag6\naV#-----------------------------------------------------------\\u000a\np166\naV#\\u000a\np167\naV# Seismograms\\u000a\np168\naV#\\u000a\np169\naV#-----------------------------------------------------------\\u000a\np170\nag6\naV# interval in time steps for writing of seismograms\\u000a\np171\naVNTSTEP_BETWEEN_OUTPUT_SEISMOS   = 4096\\u000a\np172\nag6\naV# decide if we save displacement, velocity, acceleration and/or pressure in forward runs (they can be set to true simultaneously)\\u000a\np173\naV# currently pressure seismograms are implemented in acoustic (i.e. fluid) elements only\\u000a\np174\naVSAVE_SEISMOGRAMS_DISPLACEMENT   = .true.\\u000a\np175\naVSAVE_SEISMOGRAMS_VELOCITY       = .false.\\u000a\np176\naVSAVE_SEISMOGRAMS_ACCELERATION   = .false.\\u000a\np177\naVSAVE_SEISMOGRAMS_PRESSURE       = .false.   # currently implemented in acoustic (i.e. fluid) elements only\\u000a\np178\nag6\naV# save seismograms also when running the adjoint runs for an inverse problem\\u000a\np179\naV# (usually they are unused and not very meaningful, leave this off in almost all cases)\\u000a\np180\naVSAVE_SEISMOGRAMS_IN_ADJOINT_RUN = .false.\\u000a\np181\nag6\naV# save seismograms in binary or ASCII format (binary is smaller but may not be portable between machines)\\u000a\np182\naVUSE_BINARY_FOR_SEISMOGRAMS      = .false.\\u000a\np183\nag6\naV# output seismograms in Seismic Unix format (binary with 240-byte-headers)\\u000a\np184\naVSU_FORMAT                       = .false.\\u000a\np185\nag6\naV# output seismograms in ASDF (requires asdf-library)\\u000a\np186\naVASDF_FORMAT                     = .false.\\u000a\np187\nag6\naV# decide if master process writes all the seismograms or if all processes do it in parallel\\u000a\np188\naVWRITE_SEISMOGRAMS_BY_MASTER     = .false.\\u000a\np189\nag6\naV# save all seismograms in one large combined file instead of one file per seismogram\\u000a\np190\naV# to avoid overloading shared non-local file systems such as LUSTRE or GPFS for instance\\u000a\np191\naVSAVE_ALL_SEISMOS_IN_ONE_FILE    = .false.\\u000a\np192\nag6\naV# use a trick to increase accuracy of pressure seismograms in fluid (acoustic) elements:\\u000a\np193\naV# use the second derivative of the source for the source time function instead of the source itself,\\u000a\np194\naV# and then record -potential_acoustic() as pressure seismograms instead of -potential_dot_dot_acoustic();\\u000a\np195\naV# this is mathematically equivalent, but numerically significantly more accurate because in the explicit\\u000a\np196\naV# Newmark time scheme acceleration is accurate at zeroth order while displacement is accurate at second order,\\u000a\np197\naV# thus in fluid elements potential_dot_dot_acoustic() is accurate at zeroth order while potential_acoustic()\\u000a\np198\naV# is accurate at second order and thus contains significantly less numerical noise.\\u000a\np199\naVUSE_TRICK_FOR_BETTER_PRESSURE   = .false.\\u000a\np200\nag6\naV#-----------------------------------------------------------\\u000a\np201\naV#\\u000a\np202\naV# Source encoding\\u000a\np203\naV#\\u000a\np204\naV#-----------------------------------------------------------\\u000a\np205\nag6\naV# (for acoustic simulations only for now) determines source encoding factor +/-1 depending on sign of moment tensor\\u000a\np206\naV# (see e.g. Krebs et al., 2009. Fast full-wavefield seismic inversion using encoded sources, Geophysics, 74 (6), WCC177-WCC188.)\\u000a\np207\naVUSE_SOURCE_ENCODING             = .false.\\u000a\np208\nag6\naV#-----------------------------------------------------------\\u000a\np209\naV#\\u000a\np210\naV# Energy calculation\\u000a\np211\naV#\\u000a\np212\naV#-----------------------------------------------------------\\u000a\np213\nag6\naV# to plot energy curves, for instance to monitor how CPML absorbing layers behave;\\u000a\np214\naV# should be turned OFF in most cases because a bit expensive\\u000a\np215\naVOUTPUT_ENERGY                   = .false.\\u000a\np216\naV# every how many time steps we compute energy (which is a bit expensive to compute)\\u000a\np217\naVNTSTEP_BETWEEN_OUTPUT_ENERGY    = 10\\u000a\np218\nag6\naV#-----------------------------------------------------------\\u000a\np219\naV#\\u000a\np220\naV# Adjoint kernel outputs\\u000a\np221\naV#\\u000a\np222\naV#-----------------------------------------------------------\\u000a\np223\nag6\naV# interval in time steps for reading adjoint traces\\u000a\np224\naV# 0 = read the whole adjoint sources at the same time\\u000a\np225\naVNTSTEP_BETWEEN_READ_ADJSRC      = 0\\u000a\np226\nag6\naV# read adjoint sources using ASDF (requires asdf-library)\\u000a\np227\naVREAD_ADJSRC_ASDF                = .false.\\u000a\np228\nag6\naV# this parameter must be set to .true. to compute anisotropic kernels\\u000a\np229\naV# in crust and mantle (related to the 21 Cij in geographical coordinates)\\u000a\np230\naV# default is .false. to compute isotropic kernels (related to alpha and beta)\\u000a\np231\naVANISOTROPIC_KL                  = .false.\\u000a\np232\nag6\naV# compute transverse isotropic kernels (alpha_v,alpha_h,beta_v,beta_h,eta,rho)\\u000a\np233\naV# rather than fully anisotropic kernels in case ANISOTROPIC_KL is set to .true.\\u000a\np234\naVSAVE_TRANSVERSE_KL              = .false.\\u000a\np235\nag6\naV# this parameter must be set to .true. to compute anisotropic kernels for\\u000a\np236\naV# cost function using velocity observable rather than displacement\\u000a\np237\naVANISOTROPIC_VELOCITY_KL         = .false.\\u000a\np238\nag6\naV# outputs approximate Hessian for preconditioning\\u000a\np239\naV#APPROXIMATE_HESS_KL             = .false. ##per Ebru do use for\\u000a\np240\naV#preconditioning\\u000a\np241\naVAPPROXIMATE_HESS_KL             = .true.\\u000a\np242\nag6\naV# save Moho mesh and compute Moho boundary kernels\\u000a\np243\naVSAVE_MOHO_MESH                  = .false.\\u000a\np244\nag6\naV#-----------------------------------------------------------\\u000a\np245\naV#\\u000a\np246\naV# Coupling with an injection technique (DSM, AxiSEM, or FK)\\u000a\np247\naV#\\u000a\np248\naV#-----------------------------------------------------------\\u000a\np249\nag6\naVCOUPLE_WITH_INJECTION_TECHNIQUE = .false.\\u000a\np250\naVINJECTION_TECHNIQUE_TYPE        = 3   # 1 = DSM, 2 = AxiSEM, 3 = FK\\u000a\np251\naVMESH_A_CHUNK_OF_THE_EARTH       = .false.\\u000a\np252\naVTRACTION_PATH                   = ./DATA/AxiSEM_tractions/3/\\u000a\np253\naVFKMODEL_FILE                    = FKmodel\\u000a\np254\naVRECIPROCITY_AND_KH_INTEGRAL     = .false.   # does not work yet\\u000a\np255\nag6\naV#-----------------------------------------------------------\\u000a\np256\nag6\naV# Dimitri Komatitsch, July 2014, CNRS Marseille, France:\\u000a\np257\naV# added the ability to run several calculations (several earthquakes)\\u000a\np258\naV# in an embarrassingly-parallel fashion from within the same run;\\u000a\np259\naV# this can be useful when using a very large supercomputer to compute\\u000a\np260\naV# many earthquakes in a catalog, in which case it can be better from\\u000a\np261\naV# a batch job submission point of view to start fewer and much larger jobs,\\u000a\np262\naV# each of them computing several earthquakes in parallel.\\u000a\np263\naV# To turn that option on, set parameter NUMBER_OF_SIMULTANEOUS_RUNS to a value greater than 1.\\u000a\np264\naV# To implement that, we create NUMBER_OF_SIMULTANEOUS_RUNS MPI sub-communicators,\\u000a\np265\naV# each of them being labeled "my_local_mpi_comm_world", and we use them\\u000a\np266\naV# in all the routines in "src/shared/parallel.f90", except in MPI_ABORT() because in that case\\u000a\np267\naV# we need to kill the entire run.\\u000a\np268\naV# When that option is on, of course the number of processor cores used to start\\u000a\np269\naV# the code in the batch system must be a multiple of NUMBER_OF_SIMULTANEOUS_RUNS,\\u000a\np270\naV# all the individual runs must use the same number of processor cores,\\u000a\np271\naV# which as usual is NPROC in the Par_file,\\u000a\np272\naV# and thus the total number of processor cores to request from the batch system\\u000a\np273\naV# should be NUMBER_OF_SIMULTANEOUS_RUNS * NPROC.\\u000a\np274\naV# All the runs to perform must be placed in directories called run0001, run0002, run0003 and so on\\u000a\np275\naV# (with exactly four digits).\\u000a\np276\naV#\\u000a\np277\naV# Imagine you have 10 independent calculations to do, each of them on 100 cores; you have three options:\\u000a\np278\naV#\\u000a\np279\naV# 1/ submit 10 jobs to the batch system\\u000a\np280\naV#\\u000a\np281\naV# 2/ submit a single job on 1000 cores to the batch, and in that script create a sub-array of jobs to start 10 jobs,\\u000a\np282\naV# each running on 100 cores (see e.g. http://www.schedmd.com/slurmdocs/job_array.html )\\u000a\np283\naV#\\u000a\np284\naV# 3/ submit a single job on 1000 cores to the batch, start SPECFEM3D on 1000 cores, create 10 sub-communicators,\\u000a\np285\naV# cd into one of 10 subdirectories (called e.g. run0001, run0002,... run0010) depending on the sub-communicator\\u000a\np286\naV# your MPI rank belongs to, and run normally on 100 cores using that sub-communicator.\\u000a\np287\naV#\\u000a\np288\naV# The option below implements 3/.\\u000a\np289\naV#\\u000a\np290\naVNUMBER_OF_SIMULTANEOUS_RUNS     = 1\\u000a\np291\nag6\naV# if we perform simultaneous runs in parallel, if only the source and receivers vary between these runs\\u000a\np292\naV# but not the mesh nor the model (velocity and density) then we can also read the mesh and model files\\u000a\np293\naV# from a single run in the beginning and broadcast them to all the others; for a large number of simultaneous\\u000a\np294\naV# runs for instance when solving inverse problems iteratively this can DRASTICALLY reduce I/Os to disk in the solver\\u000a\np295\naV# (by a factor equal to NUMBER_OF_SIMULTANEOUS_RUNS), and reducing I/Os is crucial in the case of huge runs.\\u000a\np296\naV# Thus, always set this option to .true. if the mesh and the model are the same for all simultaneous runs.\\u000a\np297\naV# In that case there is no need to duplicate the mesh and model file database (the content of the DATABASES_MPI\\u000a\np298\naV# directories) in each of the run0001, run0002,... directories, it is sufficient to have one in run0001\\u000a\np299\naV# and the code will broadcast it to the others)\\u000a\np300\naVBROADCAST_SAME_MESH_AND_MODEL   = .true.\\u000a\np301\nag6\naV#-----------------------------------------------------------\\u000a\np302\nag6\naV# set to true to use GPUs\\u000a\np303\naVGPU_MODE                        = .true.\\u000a\np304\nag6\naV# ADIOS Options for I/Os\\u000a\np305\naVADIOS_ENABLED                   = .false.\\u000a\np306\naVADIOS_FOR_DATABASES             = .false.\\u000a\np307\naVADIOS_FOR_MESH                  = .false.\\u000a\np308\naVADIOS_FOR_FORWARD_ARRAYS        = .false.\\u000a\np309\naVADIOS_FOR_KERNELS               = .false.\\u000a\np310\nag6\naV#**********************\\u000a\np311\naV#Serial mesh decomposer\\u000a\np312\naV#**********************\\u000a\np313\nag6\naVLTS_MODE                        = .false.\\u000a\np314\nag6\naVPARTITIONING_TYPE               = 1\\u000a\np315\na."""

In [ ]:
print(test_parfile_str == parfile_pickle_str)
#print(set(test_parfile_str.split()).difference(set(parfile_pickle_str.split())))

In [ ]:
x = goobleydoobleydoo('42')

In [ ]:
def sillyfunc(index=None):
    if index == None:
        print('none')
    else:
        print(f'index={index}')
        
sillyfunc(2)

In [ ]:
df_rec = []
for i in range(6):
    df_rec.append({'sid':i%3,'eid':0})
df = pd.DataFrame.from_records(df_rec)
print(df)
print()

print(df['sid'])
print()
s = set(df['sid'])
ss = set(df['sid'])
print(s)
print(f's == ss: {s == ss}')

In [ ]:
tmp_fqp = os.path.join(data_out_dir,'tmp')
!ls {tmp_fqp}

In [ ]:
import os
from glob import glob
for src_path in glob(data_out_dir):
    print(src_path)
    print(os.path.relpath(src_path, 'tmp'))
    '''
    os.symlink(
        os.path.relpath(
            src_path,
            'dst/'
        ),
        os.path.join('dst', os.path.basename(src_path))
    )
    '''

In [ ]:
#p = os.path.relpath('tmp', start=os.curdir)
p = os.path.relpath(tmp_fqp, start='data/')
fqpname = os.path.join(p, 'tmp')

print(p)
print(fqpname)

In [ ]:
tpath = 'data/output/tmp/TestProjects/FirstTestProject/run0001/DATA'
os.symlink('STAIONS.sid0','data/output/tmp/TestProjects/FirstTestProject/run0001/DATA/LINKSTATIONS')
!ls -ltrh {tpath}